In [2]:
import json
import pandas as pd
import numpy as np

In [3]:
f = open("../data/abcd_v1.1.json")
data = json.load(f)
data.keys()

dict_keys(['train', 'dev', 'test'])

In [5]:
# evaluation w/o concatenating consequative agent actions 
df = pd.DataFrame(columns=["context", "true_agent_response"])

for element in data["test"]:
    first_entry = True
    context = ""
    for conversation in element["original"]:
            current_speaker = conversation[0]
            if current_speaker == "customer":
                context = f"{context} {conversation[0]}: {conversation[1]}"
            
            if current_speaker == "agent" and not first_entry:
                true_agent_response = f"{conversation[1]}"
                # add row to dataframe
                new_row = {"context": context, "true_agent_response":true_agent_response}
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
                context = f"{context} {conversation[0]}: {conversation[1]}"
            
            if current_speaker == "agent" and first_entry:
                context = f"{context} {conversation[0]}: {conversation[1]}"
                first_entry = False
    

        


In [15]:
# building evaluation metric WITH concatenating consequative agent utterances
df = pd.DataFrame(columns=["context", "true_agent_response"])

for conversation in data["train"]:
    prev_speaker = conversation["original"][0][0]
    prev_utterance = ""
    context = ""
    customer_found = False
    first_entry = True

    for utterance in conversation["original"]:
        if utterance[0] == "action":
            continue
        else:
            curr_speaker = utterance[0]

            if prev_speaker == curr_speaker:
                prev_utterance = f"{prev_utterance} {utterance[1]}"
            
            else:
                
                if first_entry and prev_speaker=="agent":
                    context =  f"{context} {prev_speaker}: {prev_utterance}"
                    prev_utterance = f"{utterance[1]}"
                    prev_speaker = curr_speaker
                    first_entry = False
                
                else: 
                    if prev_speaker == "customer":
                        context =  f"{context} {prev_speaker}: {prev_utterance}"
                        customer_found = True
                    
                    if prev_speaker == "agent" and customer_found:
                        true_agent_response = f"{prev_utterance}"
                        customer_found = False
                        new_row = {"context": context, "true_agent_response": true_agent_response}
                        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

                        context = f"{context} {prev_speaker}: {prev_utterance}"
                    
                prev_speaker = curr_speaker
                prev_utterance = utterance[1]

# basically works but doesn't capture the last utterance between customer and agent          

In [16]:
df.to_excel("evaluation_concat.xlsx")

In [35]:
df.to_csv("evaluation_csv_v1.csv")

Finding Average Length of Agent Responses

In [10]:
# instead tokienize the sentence and then count the number of tokens used
df["length"] = df["true_agent_response"].str.len()

In [14]:
df["length"].mean()

52.364988290398124